# GIS Image Stack Pre-Processing 

This notebook implements a processing pipeline to transform globally available data types into spatially aligned image stacks with uniform pixel sizes. The image stack is represented as individual GeoTiff files for each data type that are cropped to an Area of Interest (AOI), and resampled at the same pixel resolution. The filesystem structure is organized as described below.

This notebook produces an output folder for each data type based on the two-letter country code (e.g., 'PK' for Pakistan), and within that folder are four GeoTiff files that represent the four processing stages implemented in this notebook (1 to 4). The forth file is the final output GeoTiff which represents the cropped, projected, and resampled raster data that constitutes a single layer in the image stack. 


## File System Structure

The top level file structure is shown below which includes five folders and three notebooks.

There is a folder for each data type at the same level as this notebook <code>prep_geospatial_data.ipynb</code>. With the exception of the Rainfall data which has a folder structure for the specified country, the other data types contain a global GeoTiff file directly within the data type folder that is used as input for this notebook. 

For all data types, output folders are created that indicate the two-letter country code. The output files from this notebook are cropped, projected, and resampled GeoTiff files that constitute a single layer in the image stack. These files will be used to create AOI image tiles using a separate notebook <code>prep_aoi_image_tiles.ipynb</code>

<pre style="font-family: monospace;">
<span style="color: gray;">./AOI         # AOI Image Stacks and Image Tiles</span>  
<span style="color: gray;">./DHS         # DHS survey data</span>
<span style="color: blue;">./gist_utils</span>  <span style="color: gray;"># Python package with convenience functions</span>
<span style="color: blue;">./Nightlights</span>
<span style="color: blue;">./Population</span>
<span style="color: blue;">./Rainfall</span>

<span style="color: gray;">./prep_aoi_image_tiles.ipynb</span>
<span style="color: blue;">./prep_geospatial_data.ipynb (this notebook)</span>
<span style="color: gray;">./prep_rainfall_gpm.ipynb</span>
</pre>

## **Input**

The input raster data for most data types is represented as a global GeoTiff file which can be cropped directly as for Nightslights and Population. However, for Rainfall, a separate notebook (`prep_rainfall_gmp.ipynb`) is first used to create daily average rainfall for the region of interest, and therefore, we need to specify the two-letter country code for those input files.

<pre style="font-family: monospace;">
./Nightlights/
    VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300_median.tif
        
./Population/
    landscan-global-2022.tif
            
./Rainfall/
    GPM_2001-2022/PK/AOI_crop_daily/GPM_2001-2022.01.V07B_PK_avg.tif
    :
    :
    GPM_2001-2022/SN/AOI_crop_daily/GPM_2001-2022.01.V07B_SN_avg.tif
    GPM_2001-2022/TD/AOI_crop_daily/GPM_2001-2022.01.V07B_TD_avg.tif
</pre>

## **Output**

The following file structure will be created by this notebook.

<pre style="font-family: monospace;">
./Nightlights/
    output/PK/
        N_VNL_v22_npp-j01_2022_global_vcmslcfg_median_PK_4_resampled_average.tif

./Population/
    output/PK/
        P_landscan-global-2022_PK_4_resampled_average.tif
            
./Rainfall/
    output/PK/  
        R_GPM_2022_PK_avg_PK_4_resampled_bilinear.tif
</pre>


## Required Configurations

The following configurations are required for each execution of this notebook: the two-letter country code and a data type. The country code impliciltly defines the latitude and longitude bounds for the AOI.

This notebook must be executed once for each available data type. 

<pre style="font-family: monospace;">
<span style="color: blue;">country_code = 'PK' </span>  # Set the country code
<span style="color: blue;">data_type    = 'N'  </span>  # Set data type (N: Nightlights, P: Population, R: Rainfall)

In [56]:
import os
from dataclasses import dataclass

# Import module that contains several convenience functions (e.g., gdal wrappers)
from gist_utils import *
from gist_utils.aoi_configurations import aoi_configurations

# Adding path to gdal commands for local system
os.environ['PATH'] += ':/Users/billk/miniforge3/envs/py39-pt/bin/' 

## 1 Set Country Code and Define AOI

The only input settings required in this notebook are the two-letter country code and the data type. Use the following single character keys to specificy each data type.

```
N: Nightlights
P: Population
R: Rainfall
```

The AOI for the specified country will be automatically computed based on the bounding box for the country plus an added buffer to allow image tiles near the boarders to be cropped. The notebook should be execuetd once for each data type.

In [93]:
#-------------------------------------------------
# REQUIRED CONFIGURATIONS HERE
#-------------------------------------------------
country_code = 'TD'   # Set the country code
data_type    = 'R'    # Set data type
proj_crs     = 'LAEA' # 'LAEA' or 'MOLL'

crs_lat = aoi_configurations[country_code]['crs_lat']
crs_lon = aoi_configurations[country_code]['crs_lon']

if proj_crs == 'MOLL':
    proj_string = '+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
elif proj_crs == 'LAEA':
    proj_string = f'+proj=laea +lat_0={crs_lat} +lon_0={crs_lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
else:
    print(f"*** Error: Invalid CRS specification '{proj_crs}'.")
    
lat_north = aoi_configurations[country_code]['lat_north']
lat_south = aoi_configurations[country_code]['lat_south']
lon_west  = aoi_configurations[country_code]['lon_west']
lon_east  = aoi_configurations[country_code]['lon_east']

case = country_code

print(proj_string)

+proj=laea +lat_0=14.0 +lon_0=18.0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs


## 2 Configure Data Type

This notebook is intended to be executed for a single data type at a time. Each of the data classes below are pre-configured. Based on the data type specified in the previous code cell, each execution of this notebook  will produce intermediate output (*.tif ) files leading up to the final processed GeoTiff.

In [94]:
# @dataclass(frozen=True)
# class DatasetConfig_Flood:                  
#     INPUT_TIF:    str  = "./dfo_3696_from_20100727_to_20101115_band5.tif"     
#     OUT_DIR:      str  = "./Flood/output/{country_code}/"    # Output folder to store processed data     
#     NODATA_SRC:   int  = -999                                          
#     NODATA_SET:   int  = -999                                          
#     RESAMPLE_ALG: str  = 'mode'
#     GDAL_INFO:    bool = True
    
#     def get_input_tif(self):
#         return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE)
    
#     def get_out_dir(self):
#         return self.OUT_DIR.format(country_code=self.COUNTRY_CODE)

In [95]:
# @dataclass(frozen=True)
# class DatasetConfig_Landcover:                  
#     INPUT_TIF:    str  = "./esalc_2020.tif"      
#     OUT_DIR:      str  = "./Landcover/output/{country_code}/"  
#     NODATA_SRC:   int  = -999                                                  
#     NODATA_SET:   int  = -999  
#     RESAMPLE_ALG: str  = 'mode'     
#     GDAL_INFO:    bool = False
        
#     def get_input_tif(self):
#         return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE)
    
#     def get_out_dir(self):
#         return self.OUT_DIR.format(country_code=self.COUNTRY_CODE)

In [96]:
@dataclass(frozen=True)
class DatasetConfig_Nightlights:
    COUNTRY_CODE: str  
    INPUT_TIF:    str = "./Nightlights/VNL_v22_npp-j01_2022_global_vcmslcfg_median.tif"  # Source data GeoTiff 
    OUT_DIR:      str = "./Nightlights/output/{country_code}/"                # Output folder to store processed data
    NODATA_SRC:   int = None                                                            # NoData value in src data                                                              # NoData value to fill in processed data
    NODATA_SET:   int = -999                                                             
    RESAMPLE_ALG: str = 'bilinear'
    GDAL_INFO:   bool = True
            
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE)

In [97]:
@dataclass(frozen=True)
class DatasetConfig_Population:
    COUNTRY_CODE: str  
    INPUT_TIF:    str = "./Population/landscan-global-2022.tif"           # Source data GeoTiff
    OUT_DIR:      str = "./Population/output/{country_code}/"  # Output folder to store processed dataNODATA_SRC:   int = -2147483647                                       # NoData value in src data  
    NODATA_SRC:   int = -2147483647  
    NODATA_SET:   int = -999                                              # NoData value to fill in processed data
    RESAMPLE_ALG: str = 'nearest'
    GDAL_INFO:    bool = True
        
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE)

In [98]:
@dataclass(frozen=True)
class DatasetConfig_Rainfall:
    COUNTRY_CODE: str  
    #INPUT_TIF:    str   = "./Rainfall/Chirps_2023/{country_code}/AOI_crop_daily/chirps-v2.0.2023_{country_code}_avg.tif"
    INPUT_TIF:    str   = "./Rainfall/GPM_2001-2022/{country_code}/AOI_crop_daily/GPM_2001-2022.01.V07B_{country_code}_avg.tif"
    OUT_DIR:      str   = "./Rainfall/output/{country_code}/"           
    NODATA_SRC:   int   = -999   # Rainfall data already pre-processed (no need to reset NoData)
    NODATA_SET:   int   = -999   # This condition (NODATA_SRC = NODATA_SET) by-passes the setting of NoData
    RESAMPLE_ALG: str   = 'bilinear'
    GDAL_INFO:    bool  = True
          
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE)

## 3 Configure AOI

The AOI data class below is used to configure the AOI for all the data types which serves to specify the cropped region for the image stack along with the re-sampling size in meters. 

In [99]:
@dataclass(frozen=True)
class AOIConfig:
    LAT_NORTH:   float                      # Define max latitude for AOI
    LAT_SOUTH:   float                      # Define min latitude for AOI
    LON_WEST:    float                      # Define min longitude for AOI
    LON_EAST:    float                      # Define max longitude for AOI
    RESAMPLE:    float = 400                # Resample data x & y (meters)
    BUF_DEG:     int = 1.0                  # Extended buffer beyond AOI

#-------------------------------------------------------------------
# Configure data type here (uncomment one line from the list below)
#-------------------------------------------------------------------

if data_type == 'N':
    data_config = DatasetConfig_Nightlights(COUNTRY_CODE=country_code)
elif data_type == 'P':   
    data_config = DatasetConfig_Population(COUNTRY_CODE=country_code)
elif data_type == 'R':
    data_config = DatasetConfig_Rainfall(COUNTRY_CODE=country_code)
else:
    print(f"Error: Invalid data type '{data_type}'. Valid data types are 'N' for Nightlights, 'P' for Population, and 'R' for Rainfall.")

aoi_config  = AOIConfig(LAT_NORTH=lat_north, LAT_SOUTH=lat_south, LON_WEST=lon_west, LON_EAST=lon_east)

if not os.path.exists(data_config.get_out_dir()):
    os.makedirs(data_config.get_out_dir())

### <span style="color: cornflowerblue;">Information Summary: Source Data</span>

In [100]:
if data_config.GDAL_INFO:
    run_gdalinfo(data_config.get_input_tif())

Driver: GTiff/GeoTIFF
Files: ./Rainfall/GPM_2001-2022/TD/AOI_crop_daily/GPM_2001-2022.01.V07B_TD_avg.tif
Size is 125, 180
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
   

##  4 Crop Data to AOI
Crop the source data to the specified AOI with a buffer. Adding a buffer is necessary to enable image tiles near the border to be created.

In [101]:
# Define AOI to encompass the country (+/- small buffer).
ul_lat, ul_lon = aoi_config.LAT_NORTH + aoi_config.BUF_DEG, aoi_config.LON_WEST - aoi_config.BUF_DEG
lr_lat, lr_lon = aoi_config.LAT_SOUTH - aoi_config.BUF_DEG, aoi_config.LON_EAST + aoi_config.BUF_DEG

# Print the results
print("\n")
print(f"Upper Left Lat: {ul_lat}")
print(f"Upper Left Lon: {ul_lon}")
print(f"Lower Right Lat: {lr_lat}")
print(f"Lower Right Lon: {lr_lon}")



Upper Left Lat: 24.41
Upper Left Lon: 12.47
Lower Right Lat: 6.44
Lower Right Lon: 25.0


In [102]:
# First, extract just the filename from the input TIFF path, removing the directory structure.
input_tif = data_config.get_input_tif()
input_tif_filename = os.path.basename(data_config.get_input_tif())

step = "_" + case + "_1_crop_geo.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), data_type + '_' + os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:          ", input_tif)
print("\n")
print("intermediate_tif:   ", intermediate_tif)
print("\n")

gdal_crop(input_tif, intermediate_tif, ul_lon, ul_lat, lr_lon, lr_lat, False)

input_tif:           ./Rainfall/GPM_2001-2022/TD/AOI_crop_daily/GPM_2001-2022.01.V07B_TD_avg.tif


intermediate_tif:    ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_1_crop_geo.tif


Input file size is 125, 180
0...10...20...30...40...50...60...70...80...90...100 - done.



##  5 Set NoData Value

In [103]:
input_tif = intermediate_tif 

step = "_" + case + "_2_nodata.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), data_type + '_' + os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")

temp_1_tif = os.path.join(data_config.get_out_dir(), 'population_1 temp.tif')
temp_2_tif = os.path.join(data_config.get_out_dir(), 'population_2_temp.tif')

print(temp_1_tif)
print(temp_2_tif)

if data_type == 'P':
    
    # First unset NoData
    gdal_unset_nodata(input_tif, temp_1_tif,)
    
    # Next, repalce data_config.NODATA_SRC with 0
    gdal_replace_value(temp_1_tif, temp_2_tif, src_value=data_config.NODATA_SRC, dst_value=0, debug=False)
    
    # Then explicitly set NoData
    gdal_set_nodata(temp_2_tif, intermediate_tif, None, data_config.NODATA_SET, False) 
else:
    # Replace nodata values
    gdal_set_nodata(input_tif, intermediate_tif, data_config.NODATA_SRC, data_config.NODATA_SET, False) 

input_tif:         ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_1_crop_geo.tif


intermediate_tif:  ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_2_nodata.tif


./Rainfall/output/TD/population_1 temp.tif
./Rainfall/output/TD/population_2_temp.tif


### <span style="color: cornflowerblue;">Information Summary: Cropped Data (with NoData set)</span>

In [104]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_2_nodata.tif
Size is 125, 180
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
           

In [105]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")

    # Access the CRS directly
    crs = dataset.crs

    # Print the EPSG code, if available
    if dataset.crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", dataset.crs.to_string())

Bounds:    BoundingBox(left=12.450, bottom=6.500, right=24.950, top=24.500)
EPSG Code: 4326


## 6 Project Geographic Data


In [106]:
input_tif = intermediate_tif 

step = "_" + case + "_3_crs.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), data_type + '_' + os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")

transform_to_CRS(input_tif, intermediate_tif, proj_string, debug=False)

input_tif:         ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_2_nodata.tif


intermediate_tif:  ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif




### <span style="color: cornflowerblue;">Information Summary: Projected Data</span>

In [107]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif
Size is 127, 183
Coordinate System is:
PROJCRS["unknown",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Lambert Azimuthal Equal Area",
        METHOD["Lambert Azimuthal Equal Area",
            ID["EPSG",9820]],
        PARAMETER["Latitude of natural origin",14,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",18,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["False easting",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["metre",1],
        

## Confirm the CRS of the Projected Data

In [108]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")
    
    # Access the CRS directly
    crs = dataset.crs
    
    # Print the EPSG code, if available
    if crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", crs.to_string())

Bounds:    BoundingBox(left=-614889.687, bottom=-826350.335, right=772544.322, top=1172865.601)
CRS is not an EPSG:  PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",14],PARAMETER["longitude_of_center",18],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


```
Bounds:    BoundingBox(left=1177334.011, bottom=803346.316, right=2493812.212, top=2987768.110)
CRS is not an EPSG:  

PROJCS["unknown",GEOGCS["unknown",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],

AUTHORITY["EPSG","6326"]],
PRIMEM["Greenwich",0],
UNIT["Degree",0.0174532925199433]],
PROJECTION["Mollweide"],
PARAMETER["central_meridian",0],
PARAMETER["false_easting",0],
PARAMETER["false_northing",0],
UNIT["metre",1,AUTHORITY["EPSG","9001"]],
AXIS["Easting",EAST],AXIS["Northing",NORTH]]
```

## 7 Resample Data

Resample the projected data using aoi_config.RESAMPLE which is measured in meters.

In [109]:
input_tif = intermediate_tif 

step = "_" + case + "_4_resampled_" + data_config.RESAMPLE_ALG + ".tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), data_type + '_' + os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")


# Resample the data
gdal_resample(input_tif, 
              intermediate_tif, 
              data_config.RESAMPLE_ALG, 
              aoi_config.RESAMPLE, 
              aoi_config.RESAMPLE)

input_tif:         ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif


intermediate_tif:  ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_4_resampled_bilinear.tif


Creating output file that is 3469P x 4998L.
Processing ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif [1/1] : 0Using internal nodata values (e.g. -999) for image ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif.
Copying nodata values from source ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_3_crs.tif to destination ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_4_resampled_bilinear.tif.
...10...20...30...40...50...60...70...80...90...100 - done.



### <span style="color: cornflowerblue;">Information Summary: Resampled Data</span>

In [110]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/TD/R_GPM_2001-2022.01.V07B_TD_avg_TD_4_resampled_bilinear.tif
Size is 3469, 4998
Coordinate System is:
PROJCRS["unknown",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Lambert Azimuthal Equal Area",
        METHOD["Lambert Azimuthal Equal Area",
            ID["EPSG",9820]],
        PARAMETER["Latitude of natural origin",14,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",18,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["False easting",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["me